# Aprendizaje NO supervisado, clustering K-means

La idea es agrupar en cuatro grupos (cluster) los en función de sus características de juego (portero, defensa, central, delantero). El objetivo es analizar el rendimiento individual de los jugadores y su contribución al equipo.

In [2]:
import pandas as pd
#cargamos el archivo
df = pd.read_csv('../data/jugadores_limpio.csv')